In [12]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

# All in same directory
DRIVER_PATH = 'chromedriver.exe'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)        
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    error_clicks = 0
    while (image_count < max_links_to_fetch) & (error_clicks < 30): # error clicks to stop when there are no more results to show by Google Images. You can tune the number
        scroll_to_end(wd)

        print('Starting search for Images')

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            print("Total Errors till now:", error_clicks)
            try:
                print('Trying to Click the Image')
                img.click()
                time.sleep(sleep_between_interactions)
                print('Image Click Successful!')
            except Exception:
                error_clicks = error_clicks + 1
                print('ERROR: Unable to Click the Image')
                if(results_start < number_results):
                	continue
                else:
                	break
                	
            results_start = results_start + 1

            # extract image urls    
            print('Extracting of Image URLs')
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            print('Current Total Image Count:', image_count)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
            	        
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["Xmen_movie_nightcrawler"]  #change your set of queries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,300,wd) # 200 denotes no. of images you want to download
        images_path = 'C:/Users/admin/Desktop/X-men classifier/images'
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()

<ipython-input-12-3a2054165ad8>:103: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)
<ipython-input-12-3a2054165ad8>:107: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image


<ipython-input-12-3a2054165ad8>:35: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 0
Total Errors till now: 0
Trying to Click the Image


<ipython-input-12-3a2054165ad8>:59: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
<ipython-input-12-3a2054165ad8>:72: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  load_more_button = wd.find_element_by_css_selector(".mye4qd")


Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 0
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 5
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 6
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 7
Total Errors till n

ERROR: Unable to Click the Image
Total Errors till now: 17
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 18
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 19
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 20
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 21
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 22
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 23
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 24
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 25
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 26
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 27
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 28
Trying

SUCCESS - saved https://i.ytimg.com/vi/tgaQW5UWU2c/maxresdefault.jpg - as C:/Users/admin/Desktop/X-men classifier/images\Xmen_movie_nightcrawler\a62c278439.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ9hXTz3qdsEriw-m2l_LdgqzmcF76gTSjNcw&usqp=CAU - as C:/Users/admin/Desktop/X-men classifier/images\Xmen_movie_nightcrawler\2145dc0afb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZ1z7t7gKSUVFgD0gJMPyxSpKX4Ji7GK6CWg&usqp=CAU - as C:/Users/admin/Desktop/X-men classifier/images\Xmen_movie_nightcrawler\d7d2a6a2ee.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRcK4EjD7A--kmHyUaLfSbYy0MsFaHdiMI40w&usqp=CAU - as C:/Users/admin/Desktop/X-men classifier/images\Xmen_movie_nightcrawler\6a9ae06c50.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSbC19-SK-mnUZk7VQzxU87GiYdQVqJafQ4QQ&usqp=CAU - as C:/Users/admin/Desktop/X-men classifier/images\Xmen_movie_nightcrawler\041255ec4a.jpg
SUCCESS - sa